## CUDA 共享内存：

### 1. 共享内存声明：

<img src="image_github/shared_memory.png" width="500" height="500">

<img src="image_github/shared_memory_code.png" width="500" height="500">

### 2. bank conflict：

为了提高内存读写带宽，共享内存被分割成了32个等大小的内存块，即Bank。因为一个Warp有32个线程，相当于一个线程对应一个内存Bank。因此任何跨越b个不同的内存bank的对n个地址进行读取和写入的操作可以被同时进行，这样就大大提高了整体带宽 ——可达到单独一个bank带宽的b倍。


<img src="image_github/shared_memory_example.png" width="500" height="500">

但是很多情况下，我们无法充分发挥bank的功能，以致于shared memory的带宽非常的小，这可能是因为我们遇到了bank冲突。

### 2.1 bank conflict 介绍：

当一个warp中的不同线程访问**同一个bank**中的**不同的字地址**时，就会发生bank冲突。

例如下图所示：

<img src="image_github/bank_conflict.png" width="600" height="500">

理想情况下就是不同的线程访问不同的bank，可能是规则的访问，如线程0读写bank0，线程1读写bank1，也可能是不规则的，如线程0读写bank1，线程1读写bank0。**这种同一个时刻每个bank只被最多1个线程访问的情况下不会出现Bank conflict。**特殊情况如果有**多个线程同时访问同一个bank的同一个地址的时候也不会产生Bank conflict，即broadcast。**但当多个线程同时访问同一个bank不同地址时，Bank conflict就产生了。例如线程0访问地址0，而线程1访问地址32，由于它们在同一个bank，就导致了这种冲突。


如果没有bank冲突的话，共享内存的访存速度将会非常的快，大约比全局内存的访问延迟低100多倍，但是速度没有寄存器快。然而，如果在使用共享内存时发生了bank冲突的话，性能将会降低很多很多。在最坏的情况下，即一个warp中的所有线程访问了相同bank的32个不同字地址的话，那么这32个访问操作将会全部被序列化，大大降低了内存带宽。**注意：不同warp中的线程之间不存在什么bank冲突。**

### 2.2 如何避免 bank conflict:

### 2.2.1 一维数组的共享缓存情况：

**如果线程同时访问4字节的值（整型或浮点型），步幅为1。同理，访问8字节的值（double），步幅为2。普遍情况下，奇数的步幅不会导致冲突。**

<img src="image_github/access_stride.png" width="400" height="500">

**bank conflict及解决方法：**

下图所示，为bank conflict情况：

<img src="image_github/bank_conflict_1.png" width="800" height="500">

解决方法1，block_size乘4，threadIdx.x乘4：（浪费内存）

<img src="image_github/bank_conflict_2.png" width="800" height="500">

解决方法2，

<img src="image_github/bank_conflict_3.png" width="800" height="500">

### 2.2.2 二维数组的共享缓存情况：

例如，下图为二维bank conflict情况，其中，block_dim为32，bank用列表示。

<img src="image_github/2D_bank_conflict_2.png" width="500" height="400">
<img src="image_github/2D_bank_conflict_3.png" width="300" height="300">

解决办法1，Memory Padding：二维内存为32 by 33。

<img src="image_github/2D_bank_conflict_1.png" width="500" height="400">
<img src="image_github/2D_bank_conflict_4.png" width="300" height="300">

解决方法2，Transpose matrix：

### 3.  Boundary Conditions & Shared Memory Tiling


???????

### 4. 动态分配共享内存：

<img src="image_github/dynamic_shared_memory.png" width="500" height="500">